# setup

In [1]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import cloudvolume as cv
from google.cloud import bigquery
import os
from google.oauth2 import service_account

In [2]:
import sys
sys.path.append('/Users/kperks/Documents/ell-connectome/eCREST/eCREST')

In [3]:
# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest, import_settings, get_cell_filepaths

## credentials

In [57]:
key_path = '/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/keys/krista_credentials.json'

In [58]:
#Set environment variables for your notebook
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path#'C:/Users/mpetkova/Dropbox/CREST/mariela_fish_credentials.json'
cred_str = os.environ['GOOGLE_APPLICATION_CREDENTIALS']
print(f'using credential file: {cred_str}')

using credential file: /Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/keys/krista_credentials.json


# get cloudvolume and bigquery client

In [59]:
efish_cloudvolume = cv.CloudVolume('gs://fish-ell/roi450um_seg32fb16fb_220930', progress=True)

In [63]:
bigquery_client = bigquery.Client(project='lcht-goog-connectomics')


# bigquery_client = bigquery.Client.from_service_account_json(json_credentials_path=key_path)

# convert state to segment locations

In [24]:
path2statesdir = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/json_states/todo')
path2skeletondir = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/skeletons')

In [25]:
cell_filepaths = get_cell_filepaths(path2statesdir)

In [49]:
cell_filepaths = dict()
for child in sorted(path2statesdir.iterdir()):
    if ('json' in child.name) & (child.is_file()):
        cell_filepaths[child.name.split('_')[1].split('.')[0]] = child

In [50]:
cell_filepaths

{'393433572': PosixPath('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/json_states/todo/sg2_393433572.json')}

## json from neuroglancer for 16nm VAST

In [64]:
skeleton_folder = 'MG2_presyn_partial'

with tqdm(total=len(cell_filepaths.keys())) as pbar:
        
    for k,v in cell_filepaths.items():
        pbar.update(1)
    
        with open(v) as f:# = open(path2statesdir+'/'+cellid_filename[ind])
            data = json.load(f)
    
        # find the index of the segmentation layer
        seg_layer_idx = next(i for i, layer in enumerate(data['layers']) if layer['type'] == 'segmentation')
        
        base_seg_list = data['layers'][seg_layer_idx]['segments']
        base_seg_str = ', '.join(str(x) for x in base_seg_list)
            
        
        
        print(f'working on {k}')
        df = pd.DataFrame()
    
        QUERY = """
        SELECT
            cast(objects.id as INT64) as seg_id,
            sample_voxel.x as x,
            sample_voxel.y as y,
            sample_voxel.z as z,
        FROM
            `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
        WHERE objects.id in {}
        """.format('('+base_seg_str+')')
    
        df = bigquery_client.query(QUERY).to_dataframe()
        df = df.drop_duplicates(subset=['seg_id'])

        # saveing information
        timestr = time.strftime("%Y%m%d-%H%M%S")
        savename_and_path = path2skeletondir / skeleton_folder / f'{k}_skeleton_{timestr}.csv';
        df.to_csv(savename_and_path, index=False)

print("done!")


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.98it/s]


working on 393433572


RefreshError: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})

## json from eCREST for 16nm VAST

In [9]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

vx_sizes = [16,16,30]

In [10]:
directory_path = Path(settings_dict['save_dir'])#/'Rachel' # specify the directory path

cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory

In [24]:
# cell_list = ['128800360', '128816291', '128879866', '128849661', '218170254',
#        '220398612', '135733302', '135670422', '137991336', '213528973',
#        '219284058', '220336851', '216994785', '220352133', '216949671',
#        '217026667', '215820918', '214627855', '214736845', '215835078',
#        '214707822', '215897023', '218109807', '215774633', '218095913',
#        '217027723', '215883144', '216952227', '302777004', '304015064',
#        '301677416', '301693439', '300565422', '301756099', '303953883',
#        '309693374', '299467022', '562815622', '563945132', '473493287',
#        '559349295', '558189118', '473461904', '558189299', '479232204',
#        '474621530', '475766527', '480330772', '475688080', '473460724',
#        '475765678', '473491444', '475812081', '475810650', '391083807',
#        '478085373', '479230342', '476894379', '474542263', '473444334',
#        '386363638', '387634327', '474573138', '474588143', '389845032',
#        '389798775', '389891998', '387585036', '387554847', '386393755',
#        '385357762', '388730139', '388667843', '394423863', '389889717',
#        '387647235', '392102325', '386455093', '386424384', '394439668',
#        '386470356', '386501395', '389858244', '300489438', '386501395',
#        '301648806', '300489438', '300582622', '299482327', '307465674',
#        '308626960'] # CELL TYPE RATIOS data: sg1 and sg2
cell_list = ['213590737', '559381249', '472284925', '472361842', '472282894','475781099', '386426114', '386409916', '386392158', '299392199','299439953', '298228381'] # CELL TYPE RATIOS data: mg1 and mg2
#['128770984', '132205966', '301727988', '303969144', '393325331','300474334'] # CELL TYPE RATIOS data: lg and lf 

### for only segments in a given region of each cell

In [18]:
savename_and_path = path2skeletondir  / 'MG2_presyn_partial'

dtype='apical dendrite'
for ind,cell_id in enumerate(cell_list):
    # f = open(path2statesdir+'/'+cellid_filename[ind])
    # data = json.load(f)
    crest = ecrest(settings_dict,filepath= cell_filepaths[cell_id], launch_viewer=False)
    data = crest.cell_data

    base_seg_list = [bs for bs in data['base_segments'][dtype]] # if only need segments from one cell structure #
    # [bs for dtype in data['base_segments'].keys() for bs in data['base_segments'][dtype]] #sum([i for i in data.get('base_segments').values()],[]); #
    #
    base_seg_list = ', '.join(str(x) for x in base_seg_list)
    
    # if not os.path.exists(savename_and_path):
    # print("working on:" +cellid[ind]+"..."+str(ind+1)+"/"+str(len(cellid))+"cells")
    df = pd.DataFrame()

    QUERY = """
    SELECT
        cast(objects.id as INT64) as seg_id,
        sample_voxel.x as x,
        sample_voxel.y as y,
        sample_voxel.z as z,
    FROM
        `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
    WHERE objects.id in {}
    """.format('('+base_seg_list+')')

    df = bigquery_client.query(QUERY).to_dataframe()
    df = df.drop_duplicates(subset=['seg_id'])
    # df = df[df['y']<15000]
    df.to_csv(savename_and_path / f'{cell_id}_ad.csv', index=False)

print("done!")

RefreshError: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})

### for all segments

In [9]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

vx_sizes = [16,16,30]

In [10]:

# directory_path = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/VAST/VAST_consolidate_reconstructed/json_states/pf_post') #settings_dict['save_dir'])# specify the directory path
directory_path = Path(settings_dict['save_dir'])

cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory

In [17]:
cell_list = ['386426114', '213605530', '126649726', '299404889'] # mg2 cells with partial presynaptic annotation

In [16]:
# base_seg_str

In [13]:
skeleton_folder = 'MG2_presyn_partial'
for k in cell_list:
    v = cell_filepaths[k]
# for k,v in cell_filepaths.items():
    # f = open(path2statesdir+'/'+cellid_filename[ind])
    # data = json.load(f)
    crest = ecrest(settings_dict,filepath= v, launch_viewer=False)
    data = crest.cell_data
    ctype = crest.get_ctype('manual')

    # find the index of the segmentation layer

    base_seg_list = [bs for dtype in data['base_segments'].keys() for bs in data['base_segments'][dtype]]  #sum([i for i in data.get('base_segments').values()],[]);
    base_seg_str = ', '.join(str(x) for x in base_seg_list)

    # saveing information
    savename_and_path = path2skeletondir  / skeleton_folder / f"{ctype}_{k}_skeleton.csv";

    if not os.path.exists(savename_and_path):
        # print("working on:" +cellid[ind]+"..."+str(ind+1)+"/"+str(len(cellid))+"cells")
        df = pd.DataFrame()

        QUERY = """
        SELECT
            cast(objects.id as INT64) as seg_id,
            sample_voxel.x as x,
            sample_voxel.y as y,
            sample_voxel.z as z,
        FROM
            `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
        WHERE objects.id in {}
        """.format('('+base_seg_str+')')

        df = bigquery_client.query(QUERY).to_dataframe()
        df = df.drop_duplicates(subset=['seg_id'])
        df.to_csv(savename_and_path, index=False)
print("done!")


RefreshError: ('invalid_grant: Invalid JWT Signature.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT Signature.'})

### for 4nm resolution 

In [94]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

vx_sizes = [16,16,30]

In [95]:
directory_path = Path(settings_dict['save_dir'])#/'Rachel' # specify the directory path

cell_filepaths = get_cell_filepaths(directory_path) # gets filepaths for all cells in a directory

In [96]:
cellid = ['368151283']

for ind,c in enumerate(cellid):
    f = open(cell_filepaths[c])
    data = json.load(f)

    # find the index of the segmentation layer

    base_seg_list = sum([i for i in data.get('base_segments').values()],[]);
    base_seg_str = ', '.join(str(x) for x in base_seg_list)

    # saveing information
    savename_and_path = path2skeletondir / f"pf_{c}_skeleton_4nm.csv";

    if not os.path.exists(savename_and_path):
        print("working on:" +c+"..."+str(ind+1)+"/"+str(len(cellid))+"cells")
        df = pd.DataFrame()

        QUERY = """
        SELECT
            cast(objects.id as INT64) as seg_id,
            sample_voxel.x as x,
            sample_voxel.y as y,
            sample_voxel.z as z,
        FROM
            `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
        WHERE objects.id in {}
        """.format('('+base_seg_str+')')

        df = bigquery_client.query(QUERY).to_dataframe()
        df = df.drop_duplicates(subset=['seg_id'])
        df['x']=df['x'] *16/4
        df['y']=df['y'] *16/4
        df.to_csv(savename_and_path, index=False)
print("done!")

working on:368151283...1/1cells
done!


# other

## get segment list from neuroglancer state

In [28]:
fpath = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/MolecularLayerModel_Michal/segments/')
fname = 'segments_axon.json'
with open(fpath/fname) as f:# = open(path2statesdir+'/'+cellid_filename[ind])
    data = json.load(f)
    
    # find the index of the segmentation layer
    seg_layer_idx = next(i for i, layer in enumerate(data['layers']) if layer['type'] == 'segmentation')
    
    base_seg_list = data['layers'][seg_layer_idx]['segments']

In [29]:
base_seg_list = [item for item in base_seg_list if '!' not in item]

In [30]:
json_string = json.dumps(base_seg_list, indent=4)
listname = fname.split('.')[0] + '_list.json'
# Write the JSON string to a file
with open(fpath/listname, "w") as f:
    f.write(json_string)

## query

In [13]:
# saveing information
# savename_and_path = path2skeletondir +'/' + "skeleton_"+cellid[ind]+".csv";


# print("working on:" +cellid[ind]+"..."+str(ind+1)+"/"+str(len(cellid))+"cells")
df = pd.DataFrame()

QUERY = """
SELECT
    cast(objects.id as INT64) as seg_id,
    sample_voxel.x as x,
    sample_voxel.y as y,
    sample_voxel.z as z,
FROM
    `lcht-goog-connectomics.ell_roi450um_seg32fb16fb_220930.objinfo` as objects
WHERE objects.id in {}
""".format('('+base_seg_str+')')

df = bigquery_client.query(QUERY).to_dataframe()
df = df.drop_duplicates(subset=['seg_id'])
# df.to_csv(savename_and_path, index=False)